<a href="https://www.kaggle.com/mickaelnarboni/clients-segmentation-rfm-maintenance?scriptVersionId=88724441" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Table of Contents

* [Dataframes frequency](#dataframes-frequency)
* [Clusters stability](#clusters-stability)
* [Adjusted Rand Index](#ari)
* [Final thoughts](#final-thoughts)


Import the relevant library for the notebook.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore") # ignore the warnings about file size
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns
from time import process_time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import math
from math import *
from sklearn.metrics import adjusted_rand_score
import plotly.io as pio
pio.renderers.default = 'iframe'
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.contrib.scatter import ScatterVisualizer
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olist-clients-segmentation/database_p4.csv
/kaggle/input/olist-clients-segmentation/geolocation_p4.csv
/kaggle/input/olist-clients-segmentation/rfm_clustering.csv


Import our dataframe. 

In [3]:
df = pd.read_csv('../input/olist-clients-segmentation/rfm_clustering.csv',sep='\t', index_col=[0], low_memory=False)
df

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
0,861eff4711a542e4b93843c6dd7febb0,474,1,1,0,124.99,0,1.078731,-0.574149,0.163933,3,Gold
1,290c77bc529b7ac935b93aa66c333dc3,233,0,1,0,289.00,0,0.283691,-0.574149,0.981946,3,Gold
2,060e732b5b29e8181a18229c7b0b2b5e,106,3,1,0,139.94,0,-0.598033,-0.574149,0.274193,3,Gold
3,259dac757896d24d7702b9acbbff3f3c,173,0,1,0,149.94,0,-0.049639,-0.574149,0.341553,3,Gold
4,345ecd01c38d18a9036ed96c73b8d066,35,3,1,0,230.00,0,-1.838550,-0.574149,0.759096,2,Silver
...,...,...,...,...,...,...,...,...,...,...,...,...
115604,1a29b476fee25c95fbafc67c5ac95cf8,148,0,1,0,74.90,0,-0.224371,-0.574149,-0.335818,0,Bronze
115605,d52a67c98be1cf6a5c84435bd38d095d,152,0,1,0,114.90,0,-0.194516,-0.574149,0.081788,3,Gold
115606,e9f50caf99f032f0bf3c55141f019d99,147,0,1,0,37.00,0,-0.231961,-0.574149,-1.024074,0,Bronze
115607,73c2643a0a458b49f58cea58833b192e,303,2,1,0,689.00,3,0.577779,-0.574149,1.829843,3,Gold


Return the shape of the dataframe. 

In [4]:
df.shape

(115609, 12)

We order the entries by Recency so it'll be easier for us to create regular interval of recency to split our customers.
We notice that our latest customer order has been fulfilled 728 days ago (almost 2 years) before downloading the Olist data.

In [6]:
df.sort_values(by=['Recency'], ascending=False, inplace=True)
df

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
99531,b7d76e111c89f7ebf14761390f0f7d17,728,1,2,0,72.89,0,1.559105,0.625897,-0.362365,0,Bronze
99530,b7d76e111c89f7ebf14761390f0f7d17,728,1,2,0,72.89,0,1.559105,0.625897,-0.362365,0,Bronze
83867,4854e9b3feff728c13ee5fc7d1547e92,728,1,1,0,59.50,0,1.559105,-0.574149,-0.560453,0,Bronze
105686,0eb1ee9dba87f5b36b4613a65074337c,700,1,1,0,100.00,0,1.515197,-0.574149,-0.053760,3,Gold
9726,61db744d2f835035a5625b59350c6b63,699,1,1,0,36.49,0,1.513596,-0.574149,-1.037620,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...
37910,b4dcade04bc548b7e3b0243c801f8c26,4,3,1,0,99.00,0,-4.266824,-0.574149,-0.063568,2,Silver
74708,7a22d14aa3c3599238509ddca4b93b01,4,3,1,0,63.90,0,-4.266824,-0.574149,-0.490828,2,Silver
60274,b701bebbdf478f5500348f03aff62121,4,3,1,0,24.90,0,-4.266824,-0.574149,-1.410589,2,Silver
49667,ff22e30958c13ffe219db7d711e8f564,0,3,2,0,290.00,0,-5.818792,0.625897,0.985317,2,Silver


<a id="dataframes-frequency"></a>
## Dataframes frequency

We split the dataframe into n = 90 number of days (business quarter) between each split based from the latest to the newest recency of clients to know until what point the model will be obsolete and need a maintenance work by Olist. 

In [5]:
n = 90

Create B0 subdataframe based on n condition.

In [ ]:
B0 = df[(df['Recency'] <= df.loc[df['Recency'] == 728]) & (df['Recency'] > df.loc[df['Recency'] == 692])]
B0

Create B1 subdataframe based on n condition.

In [ ]:
B1 = df[(df['Recency'] < B0['Recency'].iloc[-1]) & (df['Recency'] > B0['Recency'].iloc[-1] - n)]

In [7]:
B1 = df[(df['Recency'] < B0['Recency'].iloc[-1]) & (df['Recency'] > B0['Recency'].iloc[-1] - n)]
B1

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
52019,29a63a400c3ca9982907ce8de1f19527,605,1,1,0,10.90,0,1.351914,-0.574149,-2.216805,0,Bronze
59342,83e7958a94bd7f74a9414d8782f87628,605,1,4,2,38.60,0,1.351914,1.825942,-0.982759,1,Diamond
2497,728e4a7d0db4845270091ded0923c71f,605,1,3,0,20.70,0,1.351914,1.327878,-1.590876,0,Bronze
77815,f7be9bec658c62ab6240b44cd26c0b84,605,1,2,0,20.80,0,1.351914,0.625897,-1.586172,0,Bronze
77814,f7be9bec658c62ab6240b44cd26c0b84,605,1,2,0,20.80,0,1.351914,0.625897,-1.586172,0,Bronze
82016,6222ef69df52b4ed71be551725551cc0,605,1,3,0,30.70,0,1.351914,1.327878,-1.206236,0,Bronze
2498,728e4a7d0db4845270091ded0923c71f,605,1,3,0,20.70,0,1.351914,1.327878,-1.590876,0,Bronze
2496,728e4a7d0db4845270091ded0923c71f,605,1,3,0,20.70,0,1.351914,1.327878,-1.590876,0,Bronze
20157,aa811dd4c5b1ca28de6db905ec51ad5e,605,1,1,0,8.90,0,1.351914,-0.574149,-2.414636,0,Bronze
104614,fc545111108ebc9bf49eb27c828d25fd,605,1,1,0,10.90,0,1.351914,-0.574149,-2.216805,0,Bronze


Concat the two last subdataframes to create a cumulative dataframe.

In [8]:
B1 = pd.concat([B0, B1], axis=0)
B1.sort_values(by=['Recency'], ascending=False, inplace=True)
B1

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
105686,0eb1ee9dba87f5b36b4613a65074337c,700,1,1,0,100.00,0,1.515197,-0.574149,-0.053760,3,Gold
24678,87776adb449c551e74c13fc34f036105,699,1,1,0,29.99,0,1.513596,-0.574149,-1.229071,0,Bronze
71569,b8b8726af116a5cfb35b0315ecef9172,699,1,1,0,21.50,0,1.513596,-0.574149,-1.553869,0,Bronze
70861,10e89fd8e5c745f81bec101207ba4d7d,699,1,1,0,74.90,0,1.513596,-0.574149,-0.335818,0,Bronze
9726,61db744d2f835035a5625b59350c6b63,699,1,1,0,36.49,0,1.513596,-0.574149,-1.037620,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...
104268,7122b4f327033b35961170342401066c,604,1,1,0,10.49,0,1.350062,-0.574149,-2.254222,0,Bronze
45384,2e8eb06baae199a63d3a7b603efa771a,603,1,1,0,149.00,0,1.348207,-0.574149,0.335416,3,Gold
101119,e40d59fa312bc135e52952f33e30252e,603,1,1,0,370.00,0,1.348207,-0.574149,1.223074,3,Gold
85254,13c8cbdeb5bbee831faae66bcd11f9ae,603,1,2,0,798.00,3,1.348207,0.625897,1.973175,3,Gold


Create B2 subdataframe based on n condition.

In [9]:
B2 = df[(df['Recency'] < B1['Recency'].iloc[-1]) & (df['Recency'] > B1['Recency'].iloc[-1] - n)]
B2

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
39415,853ba75a0b423722ccf270eea3b4cfe4,602,1,2,0,77.00,0,1.346349,0.625897,-0.308832,3,Gold
86382,6d9e2c0d8e12764c96436cdd2cd11655,602,1,1,0,129.99,0,1.346349,-0.574149,0.202213,3,Gold
39414,853ba75a0b423722ccf270eea3b4cfe4,602,1,2,0,77.00,0,1.346349,0.625897,-0.308832,3,Gold
32158,7d08a8005756c215935d45bcc0e9a760,602,1,2,0,259.80,0,1.346349,0.625897,0.877996,3,Gold
21274,8c4eb133c4a45634289a9727130a32e1,602,1,1,0,239.90,0,1.346349,-0.574149,0.800225,3,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...
39255,51b52b687b786d0a22c5b1fbdacdc90b,514,1,1,0,79.90,0,1.169429,-0.574149,-0.272751,0,Bronze
98255,7814db11654d2bf3b43bae828b6eee05,514,1,1,0,2589.00,1,1.169429,-0.574149,3.121758,3,Gold
95195,1e7097be947ad5d0f7819bdc9d8d7d0e,514,1,1,0,2690.00,1,1.169429,-0.574149,3.159106,3,Gold
21774,1270b69469b516c18f61138f16cda7aa,514,1,2,0,37.98,0,1.169429,0.625897,-0.998562,0,Bronze


Concat the two last subdataframes to create a cumulative dataframe.

In [10]:
B2 = pd.concat([B2, B1], axis=0)
B2.sort_values(by=['Recency'], ascending=False, inplace=True)
B2

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
105686,0eb1ee9dba87f5b36b4613a65074337c,700,1,1,0,100.00,0,1.515197,-0.574149,-0.053760,3,Gold
9726,61db744d2f835035a5625b59350c6b63,699,1,1,0,36.49,0,1.513596,-0.574149,-1.037620,0,Bronze
66386,2f64e403852e6893ae37485d5fcacdaf,699,1,1,0,21.90,0,1.513596,-0.574149,-1.535879,0,Bronze
23970,7390ed59fa1febbfda31a80b4318c8cb,699,1,1,0,128.90,0,1.513596,-0.574149,0.193995,3,Gold
67562,8d3a54507421dbd2ce0a1d58046826e0,699,1,1,0,119.90,0,1.513596,-0.574149,0.123359,3,Gold
...,...,...,...,...,...,...,...,...,...,...,...,...
76125,d7b91fb85c24f413d7c4715d9464f3f2,514,1,1,0,18.25,0,1.169429,-0.574149,-1.713811,0,Bronze
7001,1421b80840e3f8f5082046e8bdc1e9ff,514,1,2,0,90.00,0,1.169429,0.625897,-0.156583,3,Gold
113010,ba4c1ae8c5e5f7f9de59ecf9723bbd97,514,1,1,0,79.00,0,1.169429,-0.574149,-0.283807,0,Bronze
59441,2e45ea9a067bfd664d3759aa47073e68,514,1,2,0,129.80,0,1.169429,0.625897,0.200785,3,Gold


Create B3 subdataframe based on n condition.

In [13]:
B3 = df[(df['Recency'] < B2['Recency'].iloc[-1]) & (df['Recency'] > B2['Recency'].iloc[-1] - n)]
B3

,CustomerUniqueID,Recency,R,Frequency,F,Monetary Value,M,Recency_standard,Frequency_standard,Monetary_value_standard,RFM Clusters,Segment
74939,2fb55b25bc8be70f9c628fb6fa5155d6,513,1,1,0,89.00,0,1.167248,-0.574149,-0.167488,0,Bronze
111481,acd09dc03842cb2b363263b562bad5fb,513,1,1,0,830.51,3,1.167248,-0.574149,2.012145,3,Gold
848,c63c0b1ccd9e3a6c8ddd1a52c4ede43d,513,1,2,0,83.80,0,1.167248,0.625897,-0.226242,3,Gold
55138,74072b8c3b18efb7d7ad672f4691a24b,513,1,1,0,49.90,0,1.167248,-0.574149,-0.732172,0,Bronze
3422,b3f022d93fd7d29a984acb2470b83791,513,1,1,0,18.99,0,1.167248,-0.574149,-1.675021,0,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...
34366,c72abd014adce3ad93d0cf7c5091e983,425,1,1,0,104.00,0,0.956572,-0.574149,-0.015483,3,Gold
11819,50fd19860348ef2a4ff4db6168e99eee,425,1,2,0,116.90,0,0.956572,0.625897,0.098630,3,Gold
25104,49f4b8e23834d6306a113014a0ae88a0,425,1,1,0,23.20,0,0.956572,-0.574149,-1.479602,0,Bronze
61124,b11721d9def930052187b45b2b59a3b4,425,1,1,0,99.00,0,0.956572,-0.574149,-0.063568,3,Gold


Concat the two last subdataframes to create a cumulative dataframe.

In [ ]:
B3 = pd.concat([B3, B2], axis=0)
B3.sort_values(by=['Recency'], ascending=False, inplace=True)
B3

Create B4 subdataframe based on n condition.

In [ ]:
B4 = df[(df['Recency'] < B3['Recency'].iloc[-1]) & (df['Recency'] > B3['Recency'].iloc[-1] - n)]
B4

Concat the two last subdataframes to create a cumulative dataframe.

In [ ]:
B4 = pd.concat([B4, B3], axis=0)
B4.sort_values(by=['Recency'], ascending=False, inplace=True)
B4

Create B5 subdataframe based on n condition.

In [ ]:
B5 = df[(df['Recency'] < B4['Recency'].iloc[-1]) & (df['Recency'] > B4['Recency'].iloc[-1] - n)]
B5

Concat the two last subdataframes to create a cumulative dataframe.

In [ ]:
B5 = pd.concat([B5, B4], axis=0)
B5.sort_values(by=['Recency'], ascending=False, inplace=True)
B5

Create B6 subdataframe based on n condition.

In [ ]:
B6 = df[(df['Recency'] < B5['Recency'].iloc[-1]) & (df['Recency'] > B5['Recency'].iloc[-1] - n)]
B6

Concat the two last subdataframes to create a cumulative dataframe.

In [ ]:
B6 = pd.concat([B6, B5], axis=0)
B6.sort_values(by=['Recency'], ascending=False, inplace=True)
B6

Create B7 subdataframe based on n condition.

In [ ]:
B7 = df[(df['Recency'] < B6['Recency'].iloc[-1]) & (df['Recency'] > B6['Recency'].iloc[-1] - n)]
B7

Concat the two last subdataframes to create a cumulative dataframe.

In [ ]:
B7 = pd.concat([B7, B6], axis=0)
B7.sort_values(by=['Recency'], ascending=False, inplace=True)
B7

Create B8 subdataframe based on n condition.

Concat the two last subdataframes to create a cumulative dataframe.

We create a dataframe for each subdataframe that will contain our three RFM variables after transformation. 

In [21]:
rfm_B0 = B0[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B1 = B1[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B2 = B2[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B3 = B3[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B4 = B4[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B5 = B5[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B6 = B6[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B7 = B7[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B8 = B8[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B9 = B9[['Recency_standard','Frequency_standard','Monetary_value_standard']]
rfm_B10 = B10[['Recency_standard','Frequency_standard','Monetary_value_standard']]

In [20]:
B0 = df.iloc[0:10000]
B1 = df.iloc[0:20000]
B2 = df.iloc[0:30000]
B3 = df.iloc[0:40000]
B4 = df.iloc[0:50000]
B5 = df.iloc[0:60000]
B6 = df.iloc[0:70000]
B7 = df.iloc[0:80000]
B8 = df.iloc[0:90000]
B9 = df.iloc[0:100000]
B10 = df.iloc[0:110000]

<a id="clusters-stability"></a>
## Clusters stability

Create a KMeans method with the same hyperparameters tested in the modeling notebook.

In [11]:
# Use the hyperparameters from our model K=4 using KMeans clustering method
kmeans_cluster = KMeans(n_clusters=4)

Create a model M(i) to fit each subdataframe B(i).

In [22]:
M0 = kmeans_cluster.fit(rfm_B0)
M1 = kmeans_cluster.fit(rfm_B1)
M2 = kmeans_cluster.fit(rfm_B2)
M3 = kmeans_cluster.fit(rfm_B3)
M4 = kmeans_cluster.fit(rfm_B4)
M5 = kmeans_cluster.fit(rfm_B5)
M6 = kmeans_cluster.fit(rfm_B6)
M7 = kmeans_cluster.fit(rfm_B7)
M8 = kmeans_cluster.fit(rfm_B8)
M9 = kmeans_cluster.fit(rfm_B9)
M10 = kmeans_cluster.fit(rfm_B10)

Create the arrays of labels of our model M0 to predict each subdataframe B(i).

In [27]:
label_00 = M0.predict(rfm_B0)
label_01 = M0.predict(rfm_B1)
label_02 = M0.predict(rfm_B2)
label_03 = M0.predict(rfm_B3)
label_04 = M0.predict(rfm_B4)
label_05 = M0.predict(rfm_B5)
label_06 = M0.predict(rfm_B6)
label_07 = M0.predict(rfm_B7)
label_08 = M0.predict(rfm_B8)
label_09 = M0.predict(rfm_B9)
label_010 = M0.predict(rfm_B10)

Create the arrays of labels of our model M(i) to fit and predict the subdataframes B(i).

In [28]:
label_11 = M1.fit_predict(rfm_B1)
label_22 = M2.fit_predict(rfm_B2)
label_33 = M3.fit_predict(rfm_B3)
label_44 = M4.fit_predict(rfm_B4)
label_55 = M5.fit_predict(rfm_B5)
label_66 = M6.fit_predict(rfm_B6)
label_77 = M7.fit_predict(rfm_B7)
label_88 = M8.fit_predict(rfm_B8)
label_99 = M9.fit_predict(rfm_B9)
label_1010 = M10.fit_predict(rfm_B10)

Now, we want to compare our model M0 that predict B(i) to each model M(i) that fit and predict M(i).

In [29]:
ari0 = adjusted_rand_score(label_00, label_00)
ari1 = adjusted_rand_score(label_01, label_11)
ari2 = adjusted_rand_score(label_02, label_22)
ari3 = adjusted_rand_score(label_03, label_33)
ari4 = adjusted_rand_score(label_04, label_44)
ari5 = adjusted_rand_score(label_05, label_55)
ari6 = adjusted_rand_score(label_06, label_66)
ari7 = adjusted_rand_score(label_07, label_77)
ari8 = adjusted_rand_score(label_08, label_88)
ari9 = adjusted_rand_score(label_09, label_99)
ari10 = adjusted_rand_score(label_010, label_1010)

<a id="ari"></a>
## Adjusted Rand Index

The ARI allows us to score our model stability to see if the predicted clustering are sorted in the same clusters as our model. The closer the score is from 1, the better.
We do that on the three variables we've been building in our model notebook: Recency, Frequency, Monetary Value.
We apply the models to each new dataframe we previously created to track the evolution of the stability along time.

We create a table that contains all our ARI scores. 

In [30]:
d = {'ARI'}
list1 = [ari0, ari1, ari2, ari3, ari4, ari5, ari6, ari7, ari8, ari9, ari10]
ari_table = pd.DataFrame(data = list1, columns=d)
ari_table

,ARI
0,1.000000
1,0.898165
2,0.979196
3,0.977536
4,0.974243
5,0.986733
6,0.990658
7,0.992080
8,0.994562
9,0.998379


Plot a curve of the ARI on each subdataframe.
We get the ARI for each new 10,000 orders based on recency from the latest to the newest.

In [31]:
pd.options.plotting.backend = "plotly"
fig = ari_table.plot(title = "RFM Predictions for clients segmentation with KMeans using ARI", labels=dict(index="10k Orders Intervals", value="Precision Score", variable=""))
fig.show()

<a id="final-thoughts"></a>
## Final thoughts